In [7]:
import urllib.request
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation

***generating image data***

In [3]:
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
file_name = "../data/horse-or-human.zip"
training_dir = '../data/horse-or-human/training/'

urllib.request.urlretrieve(url, file_name)
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [5]:
img_data_gen = ImageDataGenerator(rescale=1 / 255)

train_gen = img_data_gen.flow_from_directory(directory=training_dir, target_size=(300, 300),
                                             class_mode='binary')

Found 1027 images belonging to 2 classes.


***create model***

In [11]:
model = Sequential()

# 1st convolutional layer
model.add(Conv2D(16, kernel_size=(3, 3), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2st convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3st convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4st convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 5st convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# flatten
model.add(Flatten())

# input layer
model.add(Dense(512))
model.add(Activation('relu'))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
activation_12 (Activation)   (None, 298, 298, 16)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
activation_13 (Activation)   (None, 147, 147, 32)      0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 71, 71, 64)       

***training***

In [13]:
model.fit(train_gen, epochs=20)

Epoch 1/20
33/33 [==============================] - 42s 1s/step - loss: 0.5421 - accuracy: 0.7673

***validation***

In [14]:
val_url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip'
val_file_name = "../data/horse-or-human-val.zip"
validation_dir = '../data/horse-or-human/validation/'

urllib.request.urlretrieve(val_url, val_file_name)
zip_ref = zipfile.ZipFile(val_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [15]:
val_img_data_gen = ImageDataGenerator(rescale=1 / 255)

validation_gen = img_data_gen.flow_from_directory(directory=validation_dir, target_size=(300, 300),
                                             class_mode='binary')

Found 256 images belonging to 2 classes.


In [16]:
model.fit(train_gen, epochs=15, validation_data=validation_gen)

Epoch 1/15
33/33 [==============================] - 32s 990ms/step - loss: 3.2921e-05 - accuracy: 1.0000 - val_loss: 4.4550 - val_accuracy: 0.8047